<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%A7%91%EA%B0%92_%EC%98%88%EC%B8%A1_%EB%AC%B8%EC%A0%9C1_%ED%92%80%EC%9D%B43_%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%91%9C%EC%A4%80%ED%99%94_%EB%B0%8F_%EC%9D%B8%EC%BD%94%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 데이터 로드
import pandas as pd
house = pd.read_csv('/content/1553768847-housing.csv')

In [ ]:


house
house.info()
house.describe()




In [ ]:
# EDA
## 결측치가 많거나 불필요한 컬럼은 걸러내 보자
## 상관관계로 필요한거 솎아내보자
### 변수가 많으면 좋다 -> 모든 변수를 사용


## 이상치가 많아 보이는 컬럼은 체크해두자
## 이상치 존재 여부 확인 -> IQR

Q1 = house.quantile(0.25)
Q3 = house.quantile(0.75)

IQR = Q3 - Q1

highest = Q3 + IQR * 1.5
lowest = Q1 - IQR * 1.5


total_rooms_high = highest['total_rooms']
total_rooms_low = lowest['total_rooms']
house['total_rooms'][(house['total_rooms'] > total_rooms_high) | (house['total_rooms'] < total_rooms_low)]

## 위경도를 이용한 산점도 그리기
## 집값(median_house_value)과 다른 변수의 상관관계 확인
## 위도(longitude) 경도(latitude)를 이용해 산점도로 위치와 집값의 관계 시각적으로 확인

In [ ]:
# 데이터 전처리
## 결측치 처리
## 이상치 처리

## apply -> 데이터프레임 컬럼/ 로우를 함수에 넘겨서 처리

def is_not_outlier(row) :
  if row.sum() > 0 : # 이상치
    return False
  else : # 정상
    return True

tmp_df = (house > highest) | (house < lowest)
normal_idx = tmp_df.apply(is_not_outlier, axis=1)

house2 = house.loc[normal_idx]

In [ ]:
house3 = house2.dropna()
house3.info()

In [ ]:

house.corr()



In [71]:
house3.columns.values

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'median_house_value'], dtype=object)

In [ ]:
house3.info()

house3_num = house3.drop(['ocean_proximity','median_house_value'], axis=1)
house3_cate = house3[['ocean_proximity']]
house3_target = house3['median_house_value']


In [ ]:
house3_num
house3_cate
house3_target
## 수치형 데이터
### 표준화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(house3_num)

s_house_num = ss.transform(house3_num)


s_house_num_df = pd.DataFrame(s_house_num, columns=house3_num.columns.values)

# pd.DataFrame(s_house_num, columns=['longitude', 'latitude', 'housing_median_age',])

s_house_num_df

In [ ]:
## 범주형 데이터
house_cate_1hot = pd.get_dummies(house3_cate)
### 원핫인코딩

house_cate_1hot

In [95]:
## 범주형 데이터와 수치형 데이터 합치기

In [ ]:
# house3_num
# s_house_num_df




In [97]:


target = house3_target
train = pd.concat([s_house_num_df, house_cate_1hot.reset_index(drop=True)], axis=1)

from sklearn.model_selection import train_test_split

trd, tsd, trt, tst = train_test_split(train, target, train_size=0.8, random_state=42) # 학습용데이터는 꼭 2차원으로


In [ ]:
# 학습
## 선형회귀를 이용하여 모델 학습
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(trd, trt)






In [13]:
# 평가
## 학습한 모델을 평가하기 -> 과소적합
print(lr.score(trd, trt), lr.score(tsd, tst))

0.4769927345820525 0.45885918903846656


In [ ]:
# 예측
house.describe()['median_income']

lr.predict([[4]])

In [98]:
from sklearn.linear_model import LinearRegression

lr2 = LinearRegression()
lr2.fit(trd, trt)


LinearRegression()

In [99]:
print(lr2.score(trd, trt), lr2.score(tsd, tst))

0.6258484927545838 0.6150749466616217


In [100]:
## 피처 엔지니어링
house.columns.values

## 하이퍼 파라미터 튜닝

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'median_house_value'], dtype=object)

In [ ]:
## 다른 알고리즘 고민